In [1]:
import pandas as pd
import numpy as np
import time
df = pd.read_csv('dataForAssignment2-1.txt', delimiter= '\s+', header=None)
mean = df.mean(axis=1)
std = df.std(axis=1)
#z-normalize the data
df_normalized = df.sub(mean, axis=0).div(std, axis=0)
#data after normalization
df_normalized

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.024475,0.109122,0.158847,0.243724,0.283360,0.378893,0.428029,0.526152,0.572964,0.615191,...,-0.649395,-0.518083,-0.524342,-0.487983,-0.424895,-0.244737,-0.197167,-0.167361,-0.204449,0.039672
1,1.078714,1.096914,1.236064,1.190714,1.174257,1.223862,1.258294,1.353464,1.404458,1.336226,...,0.538691,0.694303,0.664841,0.712564,0.758851,0.709220,0.860876,0.947394,1.037807,1.129178
2,0.816179,0.945113,0.863810,1.056819,1.107537,1.216497,1.169917,1.365590,1.312019,1.355307,...,0.425592,0.390171,0.453852,0.506063,0.466518,0.689629,0.839676,0.776528,0.702730,0.841598
3,1.171243,1.199672,1.225198,1.289903,1.331749,1.304008,1.349888,1.399967,1.380264,1.425712,...,0.715199,0.666800,0.808549,0.850323,0.971796,0.982326,1.056420,1.084721,1.180924,1.145694
4,0.881996,0.887418,0.931879,0.982785,1.054900,1.051407,1.155746,1.130220,1.205432,1.248764,...,0.149719,0.253583,0.459046,0.402744,0.370815,0.477489,0.510760,0.670793,0.686973,0.786654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0.632165,0.957210,1.117498,1.263503,1.415222,1.425551,1.345437,1.391212,1.194210,1.094824,...,-1.415471,-1.330925,-1.273318,-1.097082,-0.872678,-0.694257,-0.386721,-0.115793,0.161097,0.445860
15996,1.116084,1.261604,1.372080,1.372537,1.420073,1.358158,1.234567,1.112128,0.831750,0.610557,...,-1.221618,-1.092242,-0.881380,-0.705542,-0.405001,-0.044539,0.183230,0.454914,0.671471,0.942139
15997,0.999071,1.212497,1.282585,1.349423,1.396440,1.364404,1.297555,1.144154,0.966415,0.857734,...,-1.294596,-1.123799,-1.004769,-0.817706,-0.562779,-0.255011,0.060996,0.341393,0.543129,0.852680
15998,0.898944,1.144203,1.179107,1.338276,1.397326,1.408268,1.339939,1.264861,1.098434,0.952417,...,-1.340859,-1.239739,-1.137235,-0.920603,-0.643312,-0.426272,-0.186200,0.140889,0.421685,0.604561


In [2]:
def euclidean_distance(row1, row2):
    return np.linalg.norm(row1 - row2)

In [3]:
def distance_matrix(data):
    num_rows = data.shape[0]
    distance_matrix = np.zeros((num_rows, num_rows))
    for i in range(num_rows):
        for j in range(num_rows):
            distance = euclidean_distance(data[i], data[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
    return distance_matrix

In [4]:
def calculate_cost(distance_matrix, medoids):
    """
    Calculate the total cost of a given set of medoids.

    Parameters:
    - distance_matrix: A 2D numpy array where entry (i, j) represents the
      dissimilarity (distance) between objects i and j.
    - medoids: A list of indices representing the current medoids.

    Returns:
    - total_cost: The total cost, which is the sum of distances from each point to the nearest medoid.
    """
    total_cost = 0

    # Iterate through each data point in the dataset
    for i in range(distance_matrix.shape[0]):
        # Find the minimum distance from the point to any medoid
        closest_distance = min(distance_matrix[i, medoid] for medoid in medoids)
        # Add the distance to the total cost
        total_cost += closest_distance

    return total_cost


In [10]:
"""
BUILD PHASE OF PAM algorithm

O is a 2d numpy array where the entry at (i,j) represents the dissimilarity (distance) between objects i and j.
k is the number of medoids to select.
returns a list of k medoid indices, and a list of unselected objects.
"""

def build_phase(distance_matrix, k):

  #Step 1: Initialize S with an object that has the minimal sum of distances to all others.
  num_objects = distance_matrix.shape[0]
  total_distances = np.sum(distance_matrix, axis=1)
  S = [np.argmin(total_distances)]
  U = set(range(num_objects)) - set(S)

  while len(S) < k:
    max_gain = -np.inf
    best_candidate = None

    #Step 2: Consider each object in U as a candidate for inclusion
    for i in U:
      gain = 0;

      #Step 3-4: For each object j in U - {i} compute the contribution to the gain
      for j in U-{i}:
        # Calculate Dj (dissimilarity between j and closest object in S)
        Dj = min(distance_matrix[j,s] for s in S)

        #Calculte the contribution Cji= max{Dj -d(j,i), 0}
        Cji = max(Dj - distance_matrix[j,i], 0)
        gain += Cji

      #Step 5: Track the candidate that maximizes the gain
      if gain > max_gain:
        max_gain = gain
        best_candidate = i

    #Step 6: Add the best candidate to S and remove it from U
    S.append(best_candidate)
    U.remove(best_candidate)

  return S, U


In [6]:
def calculate_cost(distance_matrix, medoids):
    """
    Calculate the total cost of a given set of medoids.

    Parameters:
    - distance_matrix: A 2D numpy array where entry (i, j) represents the
      dissimilarity (distance) between objects i and j.
    - medoids: A list of indices representing the current medoids.

    Returns:
    - total_cost: The total cost, which is the sum of distances from each point to the nearest medoid.
    """
    total_cost = 0

    # Iterate through each data point in the dataset
    for i in range(distance_matrix.shape[0]):
        # Find the minimum distance from the point to any medoid
        closest_distance = min(distance_matrix[i, medoid] for medoid in medoids)
        # Add the distance to the total cost
        total_cost += closest_distance

    return total_cost


In [8]:
def swap_phase(distance_matrix, S, U):
    """
    SWAP phase of the PAM algorithm to improve the clustering.

    Parameters:
    - distance_matrix: A 2D numpy array where entry (i, j) represents the
      dissimilarity (distance) between objects i and j.
    - S: A list of selected objects (medoids).
    - U: A set of unselected objects.

    Returns:
    - S: The updated list of selected objects (medoids) after improvement.
    - U: The updated set of unselected objects.
    """
    num_objects = distance_matrix.shape[0]
    iteration = 0

    # Step 4: Repeat until no swap decreases the objective function
    while True:
        min_Tih = float('inf')
        best_swap = None
        iteration += 1

        # Step 1: Consider all pairs (i, h) ∈ S × U
        for i in S:
            for h in U:
                Tih = 0  # Total effect of swapping i with h

                # Step 2: Compute Tih by summing up Kjih for all j ∈ U - {h}
                for j in U - {h}:
                    Dj = min(distance_matrix[j, s] for s in S)  # Closest distance to j from S
                    Ej = sorted(distance_matrix[j, s] for s in S if s != i)[0] if len(S) > 1 else float('inf')  # Second closest distance

                    # Compute Kjih based on the cases
                    if distance_matrix[j, i] > Dj:
                        if distance_matrix[j, h] >= Dj:
                            Kjih = 0
                        else:
                            Kjih = distance_matrix[j, h] - Dj
                    elif distance_matrix[j, i] == Dj:
                        if distance_matrix[j, h] < Ej:
                            Kjih = distance_matrix[j, h] - Dj
                        else:
                            Kjih = Ej - Dj

                    # Add the contribution Kjih to the total effect Tih
                    Tih += Kjih

                # Track the swap pair (i, h) with the minimum Tih
                if Tih < min_Tih:
                    min_Tih = Tih
                    best_swap = (i, h)

        # Step 3: If Tih < 0, carry out the swap and update D and E for every object
        if min_Tih < 0 and best_swap is not None:
            i, h = best_swap
            S.remove(i)
            S.append(h)
            U.remove(h)
            U.add(i)
            print(f"Iteration {iteration}, Cost: {calculate_cost(distance_matrix, selected_medoids)}")
        else:
            # No further improvement possible, break the loop
            break

    return S, U


In [11]:
dist_matrix = distance_matrix(df_normalized.to_numpy())
start_time = time.time()
selected_medoids, unselected_medoids = build_phase(dist_matrix, 4)
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")
print("Selected medoids:", selected_medoids)

Execution time: 1741.7239253520966 seconds
Selected medoids: [4856, 12423, 11580, 2597]


In [12]:
start_time = time.time()
S = swap_phase(dist_matrix, selected_medoids, unselected_medoids)
end_time = time.time()
execution_time = end_time - start_time
print("Execution times:", execution_time, "seconds")
print("Improved set of medoids after SWAP phase:", selected_medoids)

Iteration 1, Cost: 57797.39993892896
Iteration 2, Cost: 57783.04926614677
Iteration 3, Cost: 57780.99441332027
Execution times: 20075.17350792885 seconds
Improved set of medoids after SWAP phase: [2597, 7429, 14714, 11407]
